In [4]:
import os
import sys
from glob import glob
import random
import string
import yaml
import pandas as pd
import root_pandas as rpd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
#plt.style.use('seaborn')
plt.style.use('ggplot')
from collections import defaultdict
import ROOT
import copy
import tensorflow.keras
import tensorflow.saved_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Normalization
from tensorflow.keras.callbacks import Callback
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import plot_model
#from tensorflow.keras.utils.vis_utils import plot_model                                                                                                                
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import model_from_json
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from pickle import load

In [5]:
with open('config.yaml','r') as conf:
    config = yaml.safe_load(conf)
mainkeys    = list(config.keys())
tag         = config.get('Tag')
pwd         = os.getcwd()
tagdir      = os.path.join(pwd,tag)

maintree    = config.get('intree')
infiledict  = config.get('infiles')
lumi = config.get('Lumi')
print(f'Lumi : {lumi} pb-1')
usenorm     = config.get('UseNormForPlots')
scale       = config.get('DoScaling')
#print(infiledict)
clskeys = list(infiledict.keys())
#print(clskeys)
featuredict = config.get('features') 
featurelist = list(featuredict.keys())
lbnfeaturelist = config.get('lbnfeatures')
signaldict = infiledict.get('Signal')
backgrounddict = infiledict.get('Background')
restbackgrounddict = infiledict.get('RestBackground')

dfs_dict = dict()
for key, val in signaldict.items():
    df_item = rpd.read_root(val[0], key=maintree)[featurelist+lbnfeaturelist]
    df_item['tag'] = 1
    xsec = val[1]
    nEvents = val[2]
    label = val[3]
    dfs_dict[key] = [df_item, xsec, nEvents, label]

for key, val in backgrounddict.items():
    df_item = rpd.read_root(val[0], key=maintree)[featurelist+lbnfeaturelist]
    df_item['tag'] = 0
    xsec = val[1]
    nEvents = val[2]
    label = val[3]
    dfs_dict[key] = [df_item, xsec, nEvents, label]

for key, val in restbackgrounddict.items():
    df_item = rpd.read_root(val[0], key=maintree)[featurelist+lbnfeaturelist]
    df_item['tag'] = 0
    xsec = val[1]
    nEvents = val[2]
    label = val[3]
    dfs_dict[key] = [df_item, xsec, nEvents, label]
    
#dfs_dict

Lumi : 300000 pb-1


In [6]:
json_file  = open(os.path.join(tagdir,"DNN_model.json"), 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
# load weights into new model                                                                                                                             
model.load_weights(os.path.join(tagdir,"DNN_model.h5"))
print("Loaded model from disk")

Loaded model from disk


In [8]:
labelhistdict = {}
labelhistdictforplot = defaultdict(list)
nbins = 50
xmin  = 0.0
xmax  = 1.0
scale = False
for sample, info in dfs_dict.items():
    #print(sample, info[3])
    pdtonp = info[0].to_numpy()
    #print(pdtonp.shape)
    X_np = pdtonp[:,:pdtonp.shape[1]-1]
    X_test_scaled = scaler.transform(X_np) if scale else X_np
    #print(X_np.shape)
    Y_np = pdtonp[:,-1]
    #print(Y_np.shape)
    x_test_lbn = X_test_scaled[:,-len(lbnfeaturelist):].reshape(-1,len(lbnfeaturelist)//4,4)
    fit_test = np.hsplit(X_test_scaled,X_test_scaled.shape[1])
    fit_test.append(x_test_lbn)
    scores    = model.predict(fit_test)
    #print(f'{sample} : {scores}')
    hname = 'pred_hist_'+sample
    hname = ROOT.TH1F(hname, "", nbins, xmin, xmax)
    preds = list(scores[:,-1]) if sample == 'muta_lnu' else list(scores[:,1])
    for item in preds:
        hname.Fill(item)
    #print(f'sample : {sample}, nRawEvents : {hname.Integral()}')
    hname.Scale(info[1]*lumi/info[2])
    #print(f' -- nLumiScaledEvents : {hname.Integral()}')
    labelhistdict[sample] = hname # for individual process 
    labelhistdictforplot[info[3]].append(hname)
    hname.SetDirectory(0)

In [ ]:
from math import sqrt
signifs = []
bdtscores = []
hsig = labelhistdict.get('muta_lnu')
for i in range(nbins):
    ibin = i+1
    #print(f'bin > {ibin}')
    nsig = hsig.Integral(ibin, nbins)
    #print(f'nSignal : {nsig}')
    nbkg = 0
    #print('Bkg -->')
    for j,(key, val) in enumerate(labelhistdict.items()):
        if j == 0 : 
            continue
        nbkg_ = val.Integral(ibin, nbins)
        #print(f' >>--- proc : {key} --> {nbkg_}')
        nbkg += nbkg_
    if nsig+nbkg == 0:
        continue
    signif = nsig/sqrt(nsig+nbkg)
    #print(f'total Bkg. {nbkg}')
    #print(f'BDT score : {hsig.GetBinCenter(ibin)} , Significance. {signif}\n')
    bdtscores.append(round(hsig.GetBinCenter(ibin),3))
    signifs.append(round(signif,3))

#print(signifs, len(signifs))   
#print(bdtscores, len(bdtscores))

plt.figure(figsize=(12,8.5))
plt.plot(bdtscores, signifs, lw=3)
plt.xlabel('DNN score', size=20)
plt.ylabel('Significance', size=20)
plt.title('s/sqrt(s+b) vs. DNN score', size=20)
plt.text(0.2, 1.5, f'max significance : {max(signifs)}\n at BDT_score : {bdtscores[signifs.index(max(signifs))]}', fontsize = 22)
plt.savefig(os.path.join(tagdir,'maxSignificance.png'),dpi=300)
plt.show()